Creates a multi-page PDF with the tags that is suitable for printing

In [3]:
import math
import os
from pysvg.structure import *
from pysvg.core import *
from pysvg.text import *
from pysvg.filter import *
from pysvg.gradient import *
from pysvg.linking import *
from pysvg.script import *
from pysvg.shape import *
from pysvg.style import *
from pysvg.builders import *

from svglib.svglib import svg2rlg
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.graphics import renderPDF

from tagfunctions import getBin, parity, hamming_12_7_encode, drawTag

# Set parameters and create multi-page PDF with tags

In [9]:
### Main parameters to set
savedir = './barcode_tag_output'
keep_svg_files = False  # the script first saves svg pages and then converts to pdf.  Whether or not to keep the individual svg files

# parityType, applies only for no hamming tags
parityType = 'even'  # even or odd.  Here, numbers will start at zero.  In decoding (11 bit), even tags have numbers 0 to 2047, Odd have numbers 2048 to 4095


# Tag Size in mm
tagSize = 3   # 3mm for honey bees


### Other parameters

# Coding type (none/hamming)
codingType = 'none'
# Size of the canvas
canvasW, canvasH = 210, 297  # A4 paper size is 210, 297
# stroke width circle
SWC = 0.05
#Distance between tags (in fuction of rC)
dst = 6.5 # 3.5

# margin for pages
margin_x = 15
margin_y = 5


## Set other values based on parameters

# 7 bits for Hamming coded IDs
# 11 bits for not encoded IDs
if codingType == 'none':
    numBits = 11
elif codingType == 'hamming':
    numBits = 7

# Number of tags in a set
maxID = int(math.pow(2,numBits))

# radius of circle
rC = tagSize/2+0.25

# radius of arcs
rA = tagSize*1.1/3

#radius of orientation semi-circle
rO = tagSize*0.6/3

#shape builder as auxiliar
oh = ShapeBuilder()

#the canvas is generated
s = Svg(width = str(canvasW)+"mm", height = str(canvasH)+"mm")
s.set_viewBox("0 0 " + str(canvasW) + " " + str(canvasH))

#Number of rows and columns, and tags per page
row = int(math.floor((canvasH-margin_y)/(dst*rC)-1))
col = int(math.floor((canvasW-margin_x)/(dst*rC)-1))
tags_per_page = row * col
print('Number of columns per page:',col)
print('Number of rows per page:',row)


Number of columns per page: 16
Number of rows per page: 24


In [10]:
# Create multiple pages of tags
svg_pages = []
for page_start in range(0, maxID, tags_per_page):
    # Create a new SVG canvas
    s = Svg(width=str(canvasW) + "mm", height=str(canvasH) + "mm")
    s.set_viewBox("0 0 " + str(canvasW) + " " + str(canvasH))

    # Draw grid
    for indRow in range(0, row):
        coordy = (indRow + 1) * dst * rC + margin_y
        s.addElement(oh.createLine(0, coordy, canvasW, coordy, strokewidth=0.25, stroke="black"))
        # add row label
        start_id = page_start + indRow * col
        end_id = min(start_id + col - 1, maxID - 1)
        if start_id>=maxID:
            continue
        label1 = Text(parityType, x=8, y=coordy-0.5)  
        label2 = Text(f"{start_id}-{end_id}", x=8, y=coordy+2.5) 
        for label in label1,label2:
            label.set_font_size(3)
            s.addElement(label)        

    for indCol in range(0, col):
        coordx = (indCol + 1) * dst * rC + margin_x
        s.addElement(oh.createLine(coordx, 0, coordx, canvasH - 10, strokewidth=0.25, stroke="black"))

    # Draw tags
    for ind in range(tags_per_page):
        tag_id = page_start + ind
        if tag_id >= maxID:
            break
        indx = ind % col + 1
        cx = indx * dst * rC + margin_x
        indy = int(math.floor(ind / col)) + 1
        cy = indy * dst * rC + margin_y
        markBin = getBin(tag_id, numBits)
        if numBits == 7:
            markBin2 = hamming_12_7_encode(markBin)
        elif numBits == 11:
            eh = parity(markBin, parityType)
            markBin2 = markBin + str(eh)
        drawTag(s, markBin2, cx, cy, rA, rO, rC, SWC)

    # Add page text
    prepend = "" if codingType=="hamming" else parityType
    t = Text(
        f"{prepend} tags {page_start} to {min(page_start + tags_per_page - 1, maxID - 1)}",
        45,
        canvasH - 7,
    )
    t.set_font_size(4)
    s.addElement(t)

    # Save the SVG for this page
    if numBits == 7:
        svg_filename = (
            savedir + f"/BeesBookTags_12_{numBits}_bits_Hamming_{tagSize}mm_tags_"
            f"{page_start}-{min(page_start + tags_per_page - 1, maxID - 1)}.svg"
        )
    elif numBits == 11:
        svg_filename = (
            savedir + f"/BeesBookTags_{numBits}_bits_{parityType}_parity_{tagSize}mm_tags_"
            f"{page_start}-{min(page_start + tags_per_page - 1, maxID - 1)}.svg"
        )    
    s.save(svg_filename, encoding="UTF-8")
    svg_pages.append(svg_filename)

# Directory to save the final PDF
if numBits == 7:
    pdf_filename = f"{savedir}/BeesBookTags_12_{numBits}_bits_Hamming_{tagSize}mm_all_tags.pdf"
elif numBits == 11:
    pdf_filename = f"{savedir}/BeesBookTags_{numBits}_bits_{parityType}_parity_{tagSize}mm_all_tags.pdf"

# Create a new PDF file
pdf_canvas = canvas.Canvas(pdf_filename, pagesize=A4)

for svg_file in svg_pages:
    # Convert each SVG to a ReportLab drawing
    drawing = svg2rlg(svg_file)
    # Render the drawing onto the PDF canvas
    renderPDF.draw(drawing, pdf_canvas, 0, 0)
    # Add a new page for the next SVG
    pdf_canvas.showPage()

# Save the final PDF
pdf_canvas.save()

# Optionally delete the SVG files
if not keep_svg_files:
    for svg_file in svg_pages:
        try:
            os.remove(svg_file)
        except OSError as e:
            print(f"Error deleting {svg_file}: {e}")

print("PDF created:", pdf_filename)

PDF created: ./barcode_tag_output/BeesBookTags_11_bits_even_parity_3mm_all_tags.pdf


# Generate DXF files for laser cutting

In [17]:
import ezdxf


# Generate DXF files for laser cutting
dxf_pages = []
for page_start in range(0, maxID, tags_per_page):
    # Set up a new DXF document
    doc = ezdxf.new('R2010')
    doc.header['$MEASUREMENT'] = 1  # Metric units
    doc.header['$INSUNITS'] = 4  # Millimeters
    msp = doc.modelspace()

    # Add a frame to represent the canvas size (i.e. A4 paper)
    msp.add_lwpolyline(
        [(0, 0), (canvasW, 0), (canvasW, canvasH), (0, canvasH)],
        close=True
    )

    # Add circles for each tag on this page
    for ind in range(tags_per_page):
        tag_id = page_start + ind
        if tag_id >= maxID:
            break
        indx = ind % col + 1
        cx = indx * dst * rC + 5
        indy = int(math.floor(ind / col)) + 1
        cy = indy * dst * rC + 5
        # Add a circle to the DXF
        msp.add_circle(center=(cx, cy), radius=rC, dxfattribs={"color": 7})
    # Save the DXF for this page
    if numBits == 7:
        dxf_filename = (
            savedir + f"/BeesBookTags_12_{numBits}_bits_Hamming_{tagSize}mm_tags_"
            f"{page_start}-{min(page_start + tags_per_page - 1, maxID - 1)}.dxf"
        )
    elif numBits == 11:
        dxf_filename = (
            savedir + f"/BeesBookTags_{numBits}_bits_{parityType}_parity_{tagSize}mm_tags_"
            f"{page_start}-{min(page_start + tags_per_page - 1, maxID - 1)}.dxf"
        )
    doc.saveas(dxf_filename)
    dxf_pages.append(dxf_filename)

print(f"DXF files created: {len(dxf_pages)} pages.")

DXF files created: 38 pages.


In [18]:
import ezdxf


# Generate DXF files for laser cutting
dxf_pages = []
for page_start in range(0, maxID, tags_per_page):
    # Set up a new DXF document
    doc = ezdxf.new('R12')
    doc.header['$MEASUREMENT'] = 1  # Metric units
    doc.header['$INSUNITS'] = 4  # Millimeters
    msp = doc.modelspace()

    # Add a frame to represent the canvas size (i.e. A4 paper)
    msp.add_polyline2d(
        [(0, 0), (canvasW, 0), (canvasW, canvasH), (0, canvasH)],
        close=True
    )

    # Add circles for each tag on this page
    for ind in range(tags_per_page):
        tag_id = page_start + ind
        if tag_id >= maxID:
            break
        indx = ind % col + 1
        cx = indx * dst * rC + 5
        indy = int(math.floor(ind / col)) + 1
        cy = indy * dst * rC + 5
        # Add a circle to the DXF
        msp.add_circle(center=(cx, cy), radius=rC, dxfattribs={"color": 7})
    # Save the DXF for this page
    if numBits == 7:
        dxf_filename = (
            savedir + f"/BeesBookTags_12_{numBits}_bits_Hamming_{tagSize}mm_tags_"
            f"{page_start}-{min(page_start + tags_per_page - 1, maxID - 1)}-R12.dxf"
        )
    elif numBits == 11:
        dxf_filename = (
            savedir + f"/BeesBookTags_{numBits}_bits_{parityType}_parity_{tagSize}mm_tags_"
            f"{page_start}-{min(page_start + tags_per_page - 1, maxID - 1)}-R12.dxf"
        )
    doc.saveas(dxf_filename)
    dxf_pages.append(dxf_filename)

print(f"DXF files created: {len(dxf_pages)} pages.")

Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for DXF R12.
Drawing units ($INSUNITS) are not exported for D

DXF files created: 38 pages.
